In [ ]:
#To run selenium, you must download the driver and add it to your path - https://www.youtube.com/watch?v=dz59GsdvUF8
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas
import random
import time
import csv

In [ ]:
# Opens the browser
browser = webdriver.Chrome()

In [ ]:
# Login
browser.get('https://www.linkedin.com/login')

In this portion of the project, we are just collecting the links to the profiles from people that are alumni of UMD

In [ ]:
#Goes to the alumni section
browser.get('https://www.linkedin.com/school/university-of-maryland/people/')

In [ ]:
#This loop scrolls to the bottom of the page but since it loads in more profiles it needs to keep running
#To my knowledge, it will only load around 900 profiles so we are just scrolling until we reach that
for i in range(1,150):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)

In [ ]:
#Once we find all the profiles, the here takes the html and puts it into beutiful soup to easily search through it
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#This block of code searches through all the anchor tags and find links the end of links on the page
#Like this - /in/amanuel-tsadik-a5444b195/
anchors = soup.find_all('a')
links = []
for i in anchors:
    if i.get('href').startswith('/in/') == True and i.get('href') not in links:
        links.append(i.get('href'))
        print('found',i.get('href'))


In [ ]:
links_csv_data = pandas.read_csv('data.csv')
links_data = links_csv_data['profile_links'].tolist()
print(f'Imported {len(links_data)} profile links')
print(f'Found {len(links)} profile links')
print(f'New exported Length is {len(set(links_data+links))}')

In [ ]:
#Removes duplicates and exports it to the data file
export_links = list(set(links_data + links))
df = pandas.DataFrame(data={"profile_links": export_links})
df.to_csv("data.csv", sep=',',index=False)

Education, skill, and job title

In [ ]:
#To just import the dataset
export_links = pandas.read_csv('data.csv')
export_links = export_links['profile_links'].tolist()

In [23]:
base = 'https://www.linkedin.com'
counter = 0
table = {}
for profile in export_links:
    jobtitlelist = []
    skill_list = []
    education_list = []
    reference_list = []
    browser.get(f'{base}{profile}')
    for i in range(1,3):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
    profile_html = browser.page_source
    profile_soup = BeautifulSoup(profile_html, 'html.parser')
    anchor_classes = 'optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--3 artdeco-button--muted inline-flex justify-center full-width align-items-center artdeco-button--fluid'
    anchor_searcher = profile_soup.find_all("a", {"class": anchor_classes})
    experience_link = ''
    skills_link = ''
    for i in anchor_searcher:
        if i['href'].find('experience') > 0:
            experience_link = i['href']
        if i['href'].find('skills') > 0:
            skills_link = i['href']
    profile_html = browser.page_source
    profile_soup = BeautifulSoup(profile_html, 'html.parser')
    section_searcher = profile_soup.find_all("section", {"class":'artdeco-card ember-view break-words pb3 mt2'})
    for section in section_searcher:
        reference = section.find("h2", {"class":'pvs-header__title text-heading-large'})
        reference_searcher = section.find_all("span", {"class":'mr1 hoverable-link-text t-bold'})
        reference_name = str(reference).split('<!-- -->')[1].split('<!-- -->')[0]
        reference_list.append(reference_name)
    if 'Experience' in reference_list:
        if len(experience_link) > 0:
            for i in range(1,3):
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
            experience_html = browser.page_source
            experience_soup = BeautifulSoup(experience_html, 'html.parser')
            experience_fields = experience_soup.find_all("div", {"class": 'pvs-entity pvs-entity--padded pvs-list__item--no-padding-when-nested'})
            for curr_box in experience_fields:
                single_search = curr_box.find_all("span", {"class": 'mr1 t-bold'})
                multi_search = curr_box.find_all("span", {"class": 'mr1 hoverable-link-text t-bold'})
                if len(multi_search) > 1:
                    for i in range(len(multi_search)):
                        if i == 0:
                            continue
                        job_title = str(multi_search[i]).split('<!-- -->')[1].split('<!-- -->')[0]
                        jobtitlelist.append(job_title)
                else:
                    for sing in single_search:
                        job_title = str(sing).split('<!-- -->')[1].split('<!-- -->')[0]
                        jobtitlelist.append(job_title)
        else:
            for section in section_searcher:
                reference = section.find("h2", {"class":'pvs-header__title text-heading-large'})
                reference_name = str(reference).split('<!-- -->')[1].split('<!-- -->')[0]
                sing_searcher = section.find_all("span", {"class":'mr1 t-bold'})
                mulit_searcher = section.find_all("span", {"class":'mr1 hoverable-link-text t-bold'})
                if reference_name == 'Experience':
                    if len(mulit_searcher) > 1:
                        for i in range(len(mulit_searcher)):
                            if i == 0:
                                continue
                            job_title = str(mulit_searcher).split('<!-- -->')[1].split('<!-- -->')[0]
                            jobtitlelist.append(job_title)
                    else:
                        for i in sing_searcher:
                            job_title = str(i).split('<!-- -->')[1].split('<!-- -->')[0]
                            jobtitlelist.append(job_title)
    else:
        jobtitlelist.append('Na')
    if 'Skills' in reference_list:
        if len(skills_link) > 0:
            browser.get(skills_link)
            for i in range(1,3):
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
            skill_html = browser.page_source
            skill_soup = BeautifulSoup(skill_html, 'html.parser')
            skill_fields = skill_soup.find_all("span", {"class": 'mr1 hoverable-link-text t-bold'})
            for i in skill_fields:
                skill = str(i).split('<!-- -->')[1].split('<!-- -->')[0]
                skill_list.append(skill)
        else:
            for section in section_searcher:
                reference = section.find("h2", {"class":'pvs-header__title text-heading-large'})
                reference_searcher = section.find_all("span", {"class":'mr1 hoverable-link-text t-bold'})
                reference_name = str(reference).split('<!-- -->')[1].split('<!-- -->')[0]
                if reference_name == 'Skills':
                    for j in reference_searcher:
                        skill = str(j).split('<!-- -->')[1].split('<!-- -->')[0]
                        skill_list.append(skill)
    else:
        skill_list.append('Na')
    if 'Education' in reference_list:
        for section in section_searcher:
            reference = section.find("h2", {"class":'pvs-header__title text-heading-large'})
            reference_searcher = section.find_all("span", {"class":'t-14 t-normal'})
            reference_name = str(reference).split('<!-- -->')[1].split('<!-- -->')[0]
            if reference_name == 'Education':
                for j in reference_searcher:
                    edu = j.find('span',{"class":'visually-hidden'})
                    education = edu.text
                    education_list.append(education)
    else:
        education_list.append('Na')
    table[counter] = {
        'id':profile,
        'skills':skill_list,
        'titles':jobtitlelist,
        'major':education_list
    }
    print(table[counter])
    counter+=1
    time.sleep(random.randint(20, 80))
    if counter == 250:
        break

{'id': '/in/cassandra-vose/', 'skills': ['Data Warehousing', 'Data Analysis', 'Business Intelligence', 'Analysis', 'Microsoft Office', 'SQL', 'Leadership', 'Data Warehousing', 'Data Analysis', 'Business Intelligence', 'Microsoft Office', 'SQL', 'Analysis', 'Leadership'], 'titles': ['Senior Manager/Data Engineer', 'Consultant'], 'major': ['Bachelor of Science, Civil & Environmental Engineering']}
{'id': '/in/samitra-harrington-3ab715141/', 'skills': [], 'titles': ['Research Ambassador'], 'major': ["Bachelor's degree, Sociology"]}
{'id': '/in/kavyadagli/', 'skills': ['Python', 'Java', 'JavaScript', 'C', 'HTML', 'Python', 'Java', 'JavaScript', 'C', 'HTML'], 'titles': ['Software Engineer Intern', 'Teaching Assistant (Object-Oriented Programming)', 'Machine Learning Intern', 'Teaching Assistant'], 'major': ['Bachelor of Science - BS, Computer Science']}
{'id': '/in/melvin-molina/', 'skills': ['Java', 'C (Programming Language)', 'Python (Programming Language)', 'Java', 'C (Programming Langua

KeyboardInterrupt: 

In [25]:
field_names = ['id', 'skills', 'titles','major'] 
with open('Full_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    for k in table:
        writer.writerow({field: table[k].get(field) or k for field in field_names})

In [13]:
skill_fields

[<span class="mr1 hoverable-link-text t-bold">
 <span aria-hidden="true"><!-- -->Project Management<!-- --></span><span class="visually-hidden"><!-- -->Project Management<!-- --></span>
 </span>,
 <span class="mr1 hoverable-link-text t-bold">
 <span aria-hidden="true"><!-- -->Engineering<!-- --></span><span class="visually-hidden"><!-- -->Engineering<!-- --></span>
 </span>,
 <span class="mr1 hoverable-link-text t-bold">
 <span aria-hidden="true"><!-- -->Microsoft Excel<!-- --></span><span class="visually-hidden"><!-- -->Microsoft Excel<!-- --></span>
 </span>,
 <span class="mr1 hoverable-link-text t-bold">
 <span aria-hidden="true"><!-- -->Communication<!-- --></span><span class="visually-hidden"><!-- -->Communication<!-- --></span>
 </span>,
 <span class="mr1 hoverable-link-text t-bold">
 <span aria-hidden="true"><!-- -->Presentations<!-- --></span><span class="visually-hidden"><!-- -->Presentations<!-- --></span>
 </span>,
 <span class="mr1 hoverable-link-text t-bold">
 <span aria-h